In [2]:
import pandas as pd
import numpy as np

In [28]:
bike1_df = pd.read_csv("data/metro-trips-2025-q1.csv")
bike2_df = pd.read_csv("data/metro-trips-2025-q2.csv")
bike3_df = pd.read_csv("data/metro-trips-2025-q3.csv")

bike_df = pd.concat([bike1_df, bike2_df, bike3_df], ignore_index=True)

bike_keep_cols = [
    "trip_id",
    "duration",
    "start_time",
    "end_time",
    "bike_type"
]

bike_clean = bike_df[bike_keep_cols].copy()

bike_clean["start_time"] = pd.to_datetime(bike_clean["start_time"], errors="coerce")
bike_clean["end_time"] = pd.to_datetime(bike_clean["end_time"], errors="coerce")

bike_clean["date"] = bike_clean["start_time"].dt.date

bike_clean.head()

,trip_id,duration,start_time,end_time,bike_type,date
0,475609834,5,2025-01-01 00:12:00,2025-01-01 00:17:00,standard,2025-01-01
1,475609846,7,2025-01-01 00:12:00,2025-01-01 00:19:00,electric,2025-01-01
2,475609903,11,2025-01-01 00:13:00,2025-01-01 00:24:00,standard,2025-01-01
3,475609904,11,2025-01-01 00:13:00,2025-01-01 00:24:00,electric,2025-01-01
4,475610048,13,2025-01-01 00:27:00,2025-01-01 00:40:00,standard,2025-01-01


In [29]:
weather_df = pd.read_csv("data/visual-crossing-weather-data.csv")

weather_keep_cols = [
    "datetime",
    "tempmax",
    "tempmin",
    "temp",
    "humidity",
    "precip",
    "precipprob",
    "windspeed",
    "conditions"
]

weather_clean = weather_df[weather_keep_cols].copy()
weather_clean["date"] = pd.to_datetime(weather_clean["datetime"], errors="coerce")
weather_clean["date"] = weather_clean["date"].dt.date

weather_clean.head()

,datetime,tempmax,tempmin,temp,humidity,precip,precipprob,windspeed,conditions,date
0,2025-01-01,64.1,42.5,51.4,81.8,0.0,0,8.0,Partially cloudy,2025-01-01
1,2025-01-02,76.5,40.2,55.9,59.0,0.0,0,8.4,Clear,2025-01-02
2,2025-01-03,68.8,44.6,53.2,66.5,0.0,0,12.3,Clear,2025-01-03
3,2025-01-04,62.1,46.2,53.2,76.4,0.0,0,9.9,Partially cloudy,2025-01-04
4,2025-01-05,73.1,43.7,57.1,49.0,0.0,0,11.9,Clear,2025-01-05


In [66]:
merged_df = bike_clean.merge(weather_clean, on="date", how="left").drop(columns='datetime')

merged_df.head()

,trip_id,duration,start_time,end_time,bike_type,date,tempmax,tempmin,temp,humidity,precip,precipprob,windspeed,conditions
0,475609834,5,2025-01-01 00:12:00,2025-01-01 00:17:00,standard,2025-01-01,64.1,42.5,51.4,81.8,0.0,0,8.0,Partially cloudy
1,475609846,7,2025-01-01 00:12:00,2025-01-01 00:19:00,electric,2025-01-01,64.1,42.5,51.4,81.8,0.0,0,8.0,Partially cloudy
2,475609903,11,2025-01-01 00:13:00,2025-01-01 00:24:00,standard,2025-01-01,64.1,42.5,51.4,81.8,0.0,0,8.0,Partially cloudy
3,475609904,11,2025-01-01 00:13:00,2025-01-01 00:24:00,electric,2025-01-01,64.1,42.5,51.4,81.8,0.0,0,8.0,Partially cloudy
4,475610048,13,2025-01-01 00:27:00,2025-01-01 00:40:00,standard,2025-01-01,64.1,42.5,51.4,81.8,0.0,0,8.0,Partially cloudy


In [114]:
# weather condition with the most total bike-share trips

merged_df.groupby('conditions')['trip_id'].count().sort_values(ascending=False).index[0]

'Partially cloudy'

In [115]:
# ^this is biased so we normalized -->

In [101]:
# on a typical day with this weather condition, how many total rides occur

condition_totals = merged_df.groupby('conditions')['trip_id'].count()
condition_days = merged_df.groupby('conditions')['date'].nunique()
proportion = condition_totals / condition_days

proportion

conditions
Clear                     1221.532468
Overcast                  1332.200000
Partially cloudy          1288.322148
Rain                      1344.000000
Rain, Overcast             929.363636
Rain, Partially cloudy    1066.100000
dtype: float64

In [ ]:
# on a typical day with this weather condition, how many total minutes/hours of riding happen

condition_duration_totals = merged_df.groupby('conditions')['duration'].sum()
condition_days = merged_df.groupby('conditions')['date'].nunique()
avg_daily_duration = condition_duration_totals / condition_days

avg_daily_duration

conditions
Clear                     39931.831169
Overcast                  39953.266667
Partially cloudy          42350.550336
Rain                      27887.000000
Rain, Overcast            25724.545455
Rain, Partially cloudy    29541.400000
dtype: float64

In [80]:
# 2025 average temperature

weather_df['temp'].mean()

63.02215568862276

In [87]:
# round temp to whole number

merged_df['temp_round'] = merged_df['temp'].round().astype(int)

# temperature with the most total bike-share trips

merged_df.groupby('temp_round')['trip_id'].count().sort_values(ascending=False).index[0]

68

In [116]:
# ^this is biased so we normalized -->

In [98]:
# total rides per temperature

temp_totals = merged_df.groupby('temp_round')['trip_id'].count()

# number of days each temperature occurred

temp_days = merged_df.groupby('temp_round')['date'].nunique()

# on a typical day at this temperature, how many total rides occur

avg_daily_trips_by_temp = temp_totals / temp_days

avg_daily_trips_by_temp.sort_values(ascending=False).index[0]


63

In [99]:
# on a typical day at this temperature, how many total minutes/hours of riding happen

temp_duration_totals = merged_df.groupby('temp_round')['duration'].sum()
temp_days = merged_df.groupby('temp_round')['date'].nunique()
avg_daily_duration_by_temp = temp_duration_totals / temp_days

avg_daily_duration_by_temp.sort_values(ascending=False).index[0]

63

In [111]:
# 2025 average windspeed

weather_df['windspeed'].mean()

11.539221556886226

In [117]:
# round windspeed to whole number

merged_df['windspeed_round'] = merged_df['windspeed'].round().astype(int)


In [118]:
# windspeed with the most total bike-share trips

merged_df.groupby('windspeed_round')['trip_id'].count().sort_values(ascending=False).index[0]

10

In [119]:
# ^this is biased so we normalized -->

In [121]:
# total rides per windspeed

windspeed_totals = merged_df.groupby('windspeed_round')['trip_id'].count()

# number of days each windspeed occurred

windspeed_days = merged_df.groupby('windspeed_round')['date'].nunique()

# on a typical day at this windspeed, how many total rides occur

avg_daily_trips_by_windspeed = windspeed_totals / windspeed_days

avg_daily_trips_by_windspeed.sort_values(ascending=False).index[0]

13

In [132]:
# percentage of days with precipitation

weather_df[weather_df['precip'] != 0].shape[0] / weather_df.shape[0]

0.12874251497005987

In [135]:
# bin days into rainy or dry

merged_df['precip_bin'] = merged_df['precip'].apply(lambda x: "Rain" if x > 0 else "No Rain")

# total rides on rainy vs. dry days

precip_totals = merged_df.groupby('precip_bin')['trip_id'].count()

# num days with or without rain

precip_days = merged_df.groupby('precip_bin')['date'].nunique()

# on a day with rain or without rain, how many total rides occur

avg_daily_trips_by_precip = precip_totals / precip_days

avg_daily_trips_by_precip


precip_bin
No Rain    1269.713693
Rain       1027.781250
dtype: float64

In [138]:
# on a day with rain or without rain, how many total minutes/hours of riding happen

precip_duration_totals = merged_df.groupby('precip_bin')['duration'].sum()
avg_daily_duration_by_precip = precip_duration_totals / precip_days

avg_daily_duration_by_precip

precip_bin
No Rain    41428.556017
Rain       28177.656250
dtype: float64